# Methodologie 

Ce notebook a pour but de présenter la méthodologie pour récupérer les datas sur le site du BOAMP.fr concernant les données des marchés publics. Toutes les datas ont été obtenus avec l'API BOAMP (https://api.gouv.fr/les-api/boamp).

## Téléchargement des données

### Première étape: Importer les libraries nécessaires.

In [1]:
# Pour le traitement des données
import pandas as pd
import numpy as np

# Pour télécharger les données
import requests

# Autres
from tqdm import tqdm
from pathlib import Path
import json
import os

# Pour la géo-localisation
import geopy
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

### Ensuite on peut préparer le téléchargement des données.

In [2]:
# Le chemin où vont être télécharger les données
path = Path().resolve()

In [11]:
# L'ensemble des paramétres qui nous intéressent

dpts = ['33', '64', '65', '31'] # Département
years = [x for x in range(2016,2022)] # Année de 2016 à 2021
famile_cat = ["DIVERS","DSP","FNS","JOUE","MAPA","inconnu"] # Catégorie de marché
type_marche = ["TRAVAUX"] # Type de marché: Travaux

In [15]:
# On télécharge les ids qui nous intéressents et on les stores dans un dictionnaire.

queries = {}

for year in years:
    for dpt in dpts:
        for cat in famile_cat:
            for typemarche in type_marche:
                url = f"http://api.dila.fr/opendata/api-boamp/annonces/search?criterion=\
                (source_cat%3Av3%20AND%20\
                type_marche%3A{typemarche}%20AND%20regiondepartement_cat%3A72%2F{dpt}%20AND%20dateparution%3A{year}%20AND%20famille_cat%3A{cat})"
                x = requests.get(url)
                try:
                    queries[f"{year}_{dpt}_{cat}_{typemarche}"] = pd.DataFrame(x.json()["item"])
                except KeyError:
                    pass
                
# puis on concat le dictionnaire pour le transformer en pandas dataframe.
id_to_download = pd.concat(queries)

### Puis on nettoie un peu le dataframe obtenu avec les IDs qui nous intéressent.

In [53]:
id_to_download.drop_duplicates(inplace=True) #On supprimer les values dupliquées

In [55]:
id_to_download.reset_index(inplace=True) # On reset l'index

In [57]:
# On extrait quelques données intéressantes

def clean_id_data(df):
    df["annee"] = df["level_0"].str.split("_",expand=True)[0] # L'Année
    df["dept"] = df["level_0"].str.split("_",expand=True)[1] # Le département
    df["cat"] = df["level_0"].str.split("_",expand=True)[2] # La catégorie
    df["marche"] = df["level_0"].str.split("_",expand=True)[3] # Le marché
    
    return df

In [58]:
id_to_download = clean_id_data(id_to_download) # On applique la fonction

In [62]:
# On finit de nettoyer le fichier
id_to_download = id_to_download.drop(columns=["level_0","level_1","id"]).rename(columns={"value":"gestion.reference.idweb"})

In [63]:
id_to_download.head()

,gestion.reference.idweb,schema,description,annee,dept,cat,marche
0,16-157491,http://schemas.journal-officiel.gouv.fr/schema...,conception et mise en oeuvre d'une campagne de...,2016,33,DIVERS,SERVICE
1,16-121410,http://schemas.journal-officiel.gouv.fr/schema...,appel à manifesttaion d'intérêt,2016,33,DIVERS,SERVICE
2,16-78911,http://schemas.journal-officiel.gouv.fr/schema...,Concession (Ou Delegation) Du Service Public D...,2016,33,DSP,SERVICE
3,16-182879,http://schemas.journal-officiel.gouv.fr/schema...,Délégation de service public du centre équestr...,2016,33,DSP,SERVICE
4,16-90624,http://schemas.journal-officiel.gouv.fr/schema...,Délégation de service public pour la Gestion d...,2016,33,DSP,SERVICE


In [64]:
id_to_download.to_csv("./id_to_clean.csv",index=0) # Enregistrement en .csv au cas où

### Ensuite on télécharge les données sur tous les IDs qui nous intéressent, un par un.

In [16]:
data = {}

for i in tqdm(id_to_download["value"].unique()):
    url = f"http://api.dila.fr/opendata/api-boamp/annonces/v230/{i}"
    y = requests.get(url)
    data[f"{i}"] = y.json()
    with open(path / f'json/{i}.txt', 'w') as outfile: # Création de json files pour chaque id pour éviter de devoir le refaire
        json.dump(data[f"{i}"], outfile)

100%|██████████| 25073/25073 [3:50:08<00:00,  1.82it/s]  


In [ ]:
#Ensuite on concat tous les fichiers json téléchargés ensemble dans un unique pandas dataframe.

data = {}

for x in os.listdir(path_json):
    if x.split(".txt")[0] in list(id_to_download["value"].unique()):
        with open(path_json / x) as f:
            i = json.load(f)
            data[f"{x.split('.txt')[0]}"] = pd.json_normalize(i)
            
df = pd.concat(data)

df.reset_index(inplace=True)

### Et on enregistre le résultat final

In [ ]:
df.to_csv("./full_data.csv",index=0) # On enregistre le fichier final.

## Nettoyage des données

In [ ]:
# On merge les datas avec les id pour récupérer certaines références comme l'année, etc.
df = df.merge(ids,on=["gestion.reference.idweb"])

In [ ]:
# On nettoie les data en gardant uniquement les bons codes postaux
df["dept"] = df["donnees.identite.cp"].astype(str).str[:2]
dpts = ['33', '64', '65', '31']
df = df[df["dept"].isin([x for x in dpts])]
df.reset_index(drop=True,inplace=True)

In [ ]:
# Fonction pour réduire le nombre de colonnes
def concat_columns(df, regex):
    """
    On passe un panda dataframe ainsi qu'un regex des colonnes que l'on souhaite réduire à une seule.
    """

    # List des colonnes à enlever
    col_to_drop = [x for x in df.filter(regex=regex)]
    
    # Concat des colonnes
    df[regex] = df[[x for x in df.filter(regex=regex).columns]].apply(
        lambda row: ''.join(row.values.astype(str)), axis=1)
    
    # Extraction des données
    df[regex] = df[regex].str.extract(r'(<(.*).xmlns)')[1]
    
    # Drop des colonnes redondates.
    df.drop(columns=col_to_drop, inplace=True)

    return df

In [ ]:
df = concat_columns(df,"donnees.procedure.typeprocedure*")
df = concat_columns(df,"gestion.reference.typeavis.famille*")
df = concat_columns(df,"gestion.reference.typeavis.perimetre*")
df = concat_columns(df,"donnees.typeorganisme*")
df = concat_columns(df,"gestion.reference.typeavis.nature.appeloffre*")
df = concat_columns(df,"gestion.reference.typeavis.statut*")
df = concat_columns(df,"gestion.indexation.criteressociauxenv.*")

In [ ]:
# List des colonnes à enlever
col_to_drop = ["level_0",
               "level_1"]

# On enlève aussi

col_to_drop.extend(df.filter(regex="donnees.renseignementscomplementaires*").columns.tolist())
col_to_drop.extend(df.filter(regex="donnees.procedure*").columns.tolist())
col_to_drop.extend(df.filter(regex="donnees.conditiondelai*").columns.tolist())
col_to_drop.extend(df.filter(regex="donnees.modif*").columns.tolist())
col_to_drop.extend(df.filter(regex="donnees.rectif*").columns.tolist())
col_to_drop.extend(df.filter(regex="donnees.conditionparticipation*").columns.tolist())
col_to_drop.extend(df.filter(regex="gestion.indexation.date*").columns.tolist())
col_to_drop.extend(df.filter(regex="gestion.indexation.deppublication*").columns.tolist())
col_to_drop.extend(df.filter(regex="gestion.nomhtml*").columns.tolist())

# On enlève les colonnes avec plus de 90% de valeurs nulles
col_to_drop.extend(df.isnull().mean().reset_index()[df.isnull().mean().reset_index()[0] > 0.90]["index"].tolist())

In [ ]:
df.drop(columns=col_to_drop,errors='ignore',inplace=True) # On drop les colonnes!

In [ ]:
col_to_clean = ["donnees.conditionrelativemarche.participationelectroniqueoui",
                "donnees.identite.agitpourautrecomptenon",
                "donnees.identite.organismeacheteurcentralnon",
                "donnees.conditionadministrative.envoielectroniqueavecoutilnon",
                "donnees.conditionrelativemarche.autresconditionspartnon",
                "donnees.conditionrelativemarche.unitemonetaireeur",
                "donnees.conditionadministrative.envoielectroniqueavecoutilnon"
               ]

# On finit de nettoyer les données
for i in col_to_clean:
    df[i] = df[i].str.extract(r'(<(.*).xmlns)')[1]

## Géolocalisation

On commence par extraire et nettoyer l'adresse complète.

In [ ]:
df["adresse_complete"] = df["donnees.identite.adresse"].str.lower()\
    .str.replace("cedex", "", regex=True)\
    .str.replace(r"(cs\s?\d*.?)", "", regex=True)\
    .str.replace("  ", " ") \
    +\
    ", " \
    + \
    df["donnees.identite.ville"].str.lower()\
    .str.replace("cedex", "", regex=True)\
    .str.replace(r"(cs\s?\d*.?)", "", regex=True)\
    .str.replace("  ", " ")

df["adresse_complete"] = df["adresse_complete"].str.replace(" , "," ").str.title()

In [ ]:
# Utilisation de openstreemap pour obtenir les coordonnées gps
geolocator = Nominatim(user_agent="https://nominatim.openstreetmap.org")
# Delay de 1s par requête pour ne pas subir de timeout
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)

In [ ]:
# Boucle sur toutes les adresses
for i in tqdm(df["adresse_complete"]):
    # Si l'on a déjà des coordonnnées GPS, on passe à l'adresse suivante
    if df[df["adresse_complete"] == i]["latitude"].isnull().all() == False: 
        pass
    else:
        try:
            if geocode(i) is None: # Si l'adresse complète ne fonctionne pas, on essaye juste le nom de la ville
                new_search = list(df[df["adresse_complete"] == i]["donnees.identite.ville"].str.lower()\
                                   .str.replace("cedex", "", regex=True)\
                                   .str.replace(r"(cs\s?\d*.?)", "", regex=True)\
                                   .str.replace("  ", " ").str.title())[0]
                location = geocode(new_search)
                df.loc[df["adresse_complete"] == i, "latitude"] = location.latitude 
                df.loc[df["adresse_complete"] == i,
                       "longtitude"] = location.longitude
                os.sleep(1)  # On attend 1s avant de continuer
            else:
                location = geocode(i)
                df.loc[df["adresse_complete"] == i, "latitude"] = location.latitude
                df.loc[df["adresse_complete"] == i,
                       "longtitude"] = location.longitude
                os.sleep(1)  # On attend 1s avant de continuer
        except AttributeError: # Si error, on passe
            pass

In [ ]:
df.to_csv(path / "data.csv") # Puis on export les Data au format csv!

## Création de la webapp

Maintenant que les données ont été téléchargés, le reste du code pour la webapp est disponible sur github à l'adresse suivante:

https://github.com/easypanda/audap_home_assignment/tree/main/webapp